<a href="https://colab.research.google.com/github/despygurl/AV-Loan_def_pred/blob/master/Loan_Def_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependency

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install dabl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 558 kB 5.2 MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import dabl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score as f1
from sklearn.metrics import accuracy_score as acc

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "


# Data Collection

In [4]:
# Load Data
df_train = pd.read_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/train_LZV4RXX.csv')
df_test = pd.read_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/test_4zJg83n.csv')

## Checking Data

In [5]:
# Data train
df_train.head(3)

,loan_id,age,education,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none,loan_default
0,1,27,1.0,Aadhar,504264,820920,2,2,0,0
1,2,48,1.0,Aadhar,728556,831444,6,2,0,0
2,3,30,2.0,VoterID,642936,826092,0,0,0,1


In [6]:
# Data test
df_test.head(3)

,loan_id,age,education,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none
0,7001,29,1.0,Aadhar,636936,768240,2,2,0
1,7002,28,1.0,Aadhar,548988,693060,3,3,0
2,7003,28,1.0,Aadhar,651756,936600,0,0,0


# Data Exploration

In [7]:
# Load data train info
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_id           7000 non-null   int64  
 1   age               7000 non-null   int64  
 2   education         6755 non-null   float64
 3   proof_submitted   7000 non-null   object 
 4   loan_amount       7000 non-null   int64  
 5   asset_cost        7000 non-null   int64  
 6   no_of_loans       7000 non-null   int64  
 7   no_of_curr_loans  7000 non-null   int64  
 8   last_delinq_none  7000 non-null   int64  
 9   loan_default      7000 non-null   int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 547.0+ KB


In [8]:
# Load data test info
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_id           3000 non-null   int64  
 1   age               3000 non-null   int64  
 2   education         2908 non-null   float64
 3   proof_submitted   3000 non-null   object 
 4   loan_amount       3000 non-null   int64  
 5   asset_cost        3000 non-null   int64  
 6   no_of_loans       3000 non-null   int64  
 7   no_of_curr_loans  3000 non-null   int64  
 8   last_delinq_none  3000 non-null   int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 211.1+ KB


In [9]:
# Checking missing values on data train
df_train.isna().sum()

loan_id               0
age                   0
education           245
proof_submitted       0
loan_amount           0
asset_cost            0
no_of_loans           0
no_of_curr_loans      0
last_delinq_none      0
loan_default          0
dtype: int64

In [10]:
# Checking missing values on data test
df_test.isna().sum()

loan_id              0
age                  0
education           92
proof_submitted      0
loan_amount          0
asset_cost           0
no_of_loans          0
no_of_curr_loans     0
last_delinq_none     0
dtype: int64

# Data Preparation

In [11]:
# Data imputation for there are missing values in both train and test data ('education')
df_train['education'] = df_train['education'].fillna(df_train['education'].mode().iloc[0])
df_test['education'] = df_test['education'].fillna(df_test['education'].mode().iloc[0])

In [12]:
# Converting values in 'education' into category type
df_train['education'] = df_train['education'].astype('category')
df_test['education'] = df_test['education'].astype('category')

In [13]:
# Encode non-numericals data into numericals
le = LabelEncoder()

df_train['proof_submitted'] = le.fit_transform(df_train['proof_submitted'])
df_test['proof_submitted'] = le.fit_transform(df_test['proof_submitted'])

## Splitting Data

In [14]:
# Splitting features and target on data train
X_train = df_train.iloc[:,:-1]
y_train = df_train['loan_default']

In [15]:
# Define data test
X_test = df_test

In [16]:
# Scaling data train and test to get better and robust data of train and test data
sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.fit_transform(X_test)

# Modelling

## AutoML with Dabl

In [17]:
fc = dabl.SimpleClassifier(random_state=0)
fc.fit(X_train_sc, y_train)

Running DummyClassifier()
accuracy: 0.600 average_precision: 0.400 roc_auc: 0.500 recall_macro: 0.500 f1_macro: 0.375
=== new best DummyClassifier() (using recall_macro):
accuracy: 0.600 average_precision: 0.400 roc_auc: 0.500 recall_macro: 0.500 f1_macro: 0.375

Running GaussianNB()
accuracy: 0.593 average_precision: 0.460 roc_auc: 0.578 recall_macro: 0.533 f1_macro: 0.513
=== new best GaussianNB() (using recall_macro):
accuracy: 0.593 average_precision: 0.460 roc_auc: 0.578 recall_macro: 0.533 f1_macro: 0.513

Running MultinomialNB()
accuracy: 0.600 average_precision: 0.446 roc_auc: 0.551 recall_macro: 0.500 f1_macro: 0.375
Running DecisionTreeClassifier(class_weight='balanced', max_depth=1)
accuracy: 0.529 average_precision: 0.428 roc_auc: 0.553 recall_macro: 0.553 f1_macro: 0.527
=== new best DecisionTreeClassifier(class_weight='balanced', max_depth=1) (using recall_macro):
accuracy: 0.529 average_precision: 0.428 roc_auc: 0.553 recall_macro: 0.553 f1_macro: 0.527

Running Decision

SimpleClassifier(random_state=0)

In [18]:
# Applying model based on Dabl recommendation: LogisticRegression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=2,class_weight='balanced', max_iter=1000).fit(X_train_sc, y_train)

## Manually search for best result

In [19]:
# Build models
c_svm = svm.SVC()
c_knn = KNeighborsClassifier(n_neighbors=3,algorithm='kd_tree')
c_nn = MLPClassifier(hidden_layer_sizes=(5,),activation='logistic',solver='adam',learning_rate='adaptive')
c_dt = DecisionTreeClassifier()

# Fitting models
c_svm = c_svm.fit(X_train_sc, y_train)
c_knn = c_knn.fit(X_train_sc, y_train)
c_nn = c_nn.fit(X_train_sc, y_train)
c_dt = c_dt.fit(X_train_sc, y_train)

# Model Evaluation

In [20]:
# Checking model evaluation based on required metrics: F1 Score
print(f"F1 Score SVM: {f1(y_train, c_svm.predict(X_train_sc), average='macro')}")
print(f"F1 Score KNN: {f1(y_train, c_knn.predict(X_train_sc), average='macro')}")
print(f"F1 Score Neural Network: {f1(y_train, c_nn.predict(X_train_sc), average='macro')}")
print(f"F1 Score Decision Tree: {f1(y_train, c_dt.predict(X_train_sc), average='macro')}")
print(f"F1 Score Logistic Regression:{f1(y_train, clf.predict(X_train_sc), average='macro')}")

F1 Score SVM: 0.4522483253644362
F1 Score KNN: 0.7471988841500439
F1 Score Neural Network: 0.46685487047185464
F1 Score Decision Tree: 1.0
F1 Score Logistic Regression:0.5710090304908797


In [21]:
# Checking model evaluation based on required metrics: F1 Score
print(f"Accuracy Score SVM: {round(acc(y_train, c_svm.predict(X_train_sc))*100,2)}")
print(f"Accuracy Score KNN: {round(acc(y_train, c_knn.predict(X_train_sc))*100,2)}")
print(f"Accuracy Score Neural Network: {round(acc(y_train, c_nn.predict(X_train_sc))*100,2)}")
print(f"Accuracy Score Decision Tree: {round(acc(y_train, c_dt.predict(X_train_sc))*100,2)}")
print(f"Accuracy Score Logistic Regression:{round(acc(y_train, clf.predict(X_train_sc))*100,2)}")

Accuracy Score SVM: 61.79
Accuracy Score KNN: 76.11
Accuracy Score Neural Network: 60.43
Accuracy Score Decision Tree: 100.0
Accuracy Score Logistic Regression:57.36


# Prediction

In [22]:
pred_svm = c_svm.predict(X_test_sc)
pred_knn = c_knn.predict(X_test_sc)
pred_nn = c_nn.predict(X_test_sc)
pred_dt = c_dt.predict(X_test_sc)
clf_pred = clf.predict(X_test_sc)

# Saving Prediction Result to CSV Files

In [24]:
sub_svm = pd.DataFrame({'loan_id':X_test['loan_id'], 'loan_default':pred_svm})
sub_knn = pd.DataFrame({'loan_id':X_test['loan_id'], 'loan_default':pred_knn})
sub_nn = pd.DataFrame({'loan_id':X_test['loan_id'], 'loan_default':pred_nn})
sub_dt = pd.DataFrame({'loan_id':X_test['loan_id'], 'loan_default':pred_dt})
sub_lr = pd.DataFrame({'loan_id':X_test['loan_id'], 'loan_default':clf_pred})

sub_svm.to_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/Submission/sub_svm.csv', index=False)
sub_knn.to_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/Submission/sub_knn.csv', index=False)
sub_nn.to_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/Submission/sub_nn.csv', index=False)
sub_dt.to_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/Submission/sub_dt.csv', index=False)
sub_lr.to_csv('/content/drive/MyDrive/3_BELAJAR/ONLINE COURSE/AnalyticsVidhya/Data Hackathon/AnalyticsVidhya - Loan Default Prediction/sub_lr.csv', index=False)